# ChessBot Research Project Playground
here is where we can play against the bot (for now against ourself)

In [4]:
import chess
import chess.svg
import chess.engine
from IPython.display import display, clear_output
import random
from time import sleep
import pickle
import numpy as np

In [11]:
def play_random_game(board):
    while not board.is_game_over():
        legal_moves = list(board.legal_moves)
        move = random.choice(legal_moves)
        board.push(move)
    return board.result()

def show_board(board):
    display(chess.svg.board(board=board,size=400))

In [9]:
class Node:
    def __init__(self, board, parent=None):
        self.board = board
        self.parent = parent
        self.children = []
        self.visits = 0
        self.wins = 0
        self.untried_moves = list(self.board.legal_moves)

    def expand(self):
        move = random.choice(self.untried_moves)
        self.untried_moves.remove(move)
        new_board = self.board.copy()
        new_board.push(move)
        new_node = Node(new_board, self)
        self.children.append(new_node)
        return new_node
        
    def select_best_child(self):
        best_score = -float('inf')
        best_child = None
        for child in self.children:
            score = child.wins/child.visits + 0.5*np.sqrt(2*np.log(self.visits)/child.visits)
            if score > best_score:
                best_score = score
                best_child = child
        return best_child

class MCTS:
    def __init__(self, chess_game, simulations):
        self.chess_game = chess_game
        self.simulations = simulations
        
    def search(self):
        root = Node(self.chess_game)
        for i in range(self.simulations):
            node = root
            while node.untried_moves == [] and node.children != []:
                node = node.select_best_child()
            if node.untried_moves != []:
                child = node.expand()
                node = child
            result = play_random_game(node.board)
            while node != None:
                node.visits += 1
                if result == node.board.turn:
                    node.wins += 1
                node = node.parent
        return max(root.children, key = lambda x: x.visits).board.pop()

    def save(self):
        with open('mcts_state.pickle', 'wb') as handle:
            print("saving MCTS")
            pickle.dump(self, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
sims_per_move = 1000
moves = 100

game = chess.Board()

mcts = MCTS(game, sims_per_move)

for i in range(moves):
    # Make the best move according to the MCTS search
    move = mcts.search()
    game.push(move)
    
    # Print the current game state
    show_board(game)

    # check if the game is over
    if game.is_game_over():
        print("Game over. Result:", game.result())
        break
    
    # save the MCTS state
    mcts.save()

AttributeError: 'Board' object has no attribute 'play_random_game'